# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 17 2022 9:07AM,239823,19,8156274,"Graystone, LLC",Released
1,Jun 17 2022 9:07AM,239823,19,8156276,"Graystone, LLC",Released
2,Jun 17 2022 8:59AM,239817,10,0085775424,ISDIN Corporation,Released
3,Jun 17 2022 8:59AM,239817,10,0085775426,ISDIN Corporation,Released
4,Jun 17 2022 8:59AM,239817,10,0085775427,ISDIN Corporation,Released
5,Jun 17 2022 8:59AM,239817,10,0085775449,ISDIN Corporation,Released
6,Jun 17 2022 8:59AM,239817,10,0085775452,ISDIN Corporation,Released
7,Jun 17 2022 8:59AM,239817,10,0085775450,ISDIN Corporation,Released
8,Jun 17 2022 8:59AM,239817,10,0085775454,ISDIN Corporation,Released
9,Jun 17 2022 8:59AM,239817,10,0085775453,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,239818,Released,21
31,239819,Released,2
32,239820,Released,1
33,239821,Released,11
34,239823,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239818,NaN,NaN,21.0
239819,NaN,NaN,2.0
239820,NaN,NaN,1.0
239821,NaN,NaN,11.0
239823,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239727,0.0,0.0,1.0
239754,0.0,0.0,1.0
239755,0.0,0.0,1.0
239758,1.0,0.0,7.0
239762,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239727,0,0,1
239754,0,0,1
239755,0,0,1
239758,1,0,7
239762,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239727,0,0,1
1,239754,0,0,1
2,239755,0,0,1
3,239758,1,0,7
4,239762,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239727,,,1
1,239754,,,1
2,239755,,,1
3,239758,1,,7
4,239762,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 17 2022 9:07AM,239823,19,"Graystone, LLC"
2,Jun 17 2022 8:59AM,239817,10,ISDIN Corporation
62,Jun 17 2022 8:59AM,239821,12,Hush Hush
73,Jun 17 2022 8:59AM,239818,15,"Person & Covey, Inc."
94,Jun 17 2022 8:58AM,239820,10,ISDIN Corporation
95,Jun 17 2022 8:57AM,239819,12,Hush Hush
97,Jun 17 2022 8:54AM,239816,15,"Virtus Pharmaceuticals, LLC"
99,Jun 17 2022 8:53AM,239815,10,"Amcyte Pharma, Inc."
100,Jun 17 2022 8:50AM,239814,15,"Person & Covey, Inc."
114,Jun 17 2022 8:45AM,239813,10,"Senseonics, Incorporated"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 17 2022 9:07AM,239823,19,"Graystone, LLC",,,2
1,Jun 17 2022 8:59AM,239817,10,ISDIN Corporation,,,60
2,Jun 17 2022 8:59AM,239821,12,Hush Hush,,,11
3,Jun 17 2022 8:59AM,239818,15,"Person & Covey, Inc.",,,21
4,Jun 17 2022 8:58AM,239820,10,ISDIN Corporation,,,1
5,Jun 17 2022 8:57AM,239819,12,Hush Hush,,,2
6,Jun 17 2022 8:54AM,239816,15,"Virtus Pharmaceuticals, LLC",,,2
7,Jun 17 2022 8:53AM,239815,10,"Amcyte Pharma, Inc.",,,1
8,Jun 17 2022 8:50AM,239814,15,"Person & Covey, Inc.",,,14
9,Jun 17 2022 8:45AM,239813,10,"Senseonics, Incorporated",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 17 2022 9:07AM,239823,19,"Graystone, LLC",2,,
1,Jun 17 2022 8:59AM,239817,10,ISDIN Corporation,60,,
2,Jun 17 2022 8:59AM,239821,12,Hush Hush,11,,
3,Jun 17 2022 8:59AM,239818,15,"Person & Covey, Inc.",21,,
4,Jun 17 2022 8:58AM,239820,10,ISDIN Corporation,1,,
5,Jun 17 2022 8:57AM,239819,12,Hush Hush,2,,
6,Jun 17 2022 8:54AM,239816,15,"Virtus Pharmaceuticals, LLC",2,,
7,Jun 17 2022 8:53AM,239815,10,"Amcyte Pharma, Inc.",1,,
8,Jun 17 2022 8:50AM,239814,15,"Person & Covey, Inc.",14,,
9,Jun 17 2022 8:45AM,239813,10,"Senseonics, Incorporated",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 17 2022 9:07AM,239823,19,"Graystone, LLC",2,,
1,Jun 17 2022 8:59AM,239817,10,ISDIN Corporation,60,,
2,Jun 17 2022 8:59AM,239821,12,Hush Hush,11,,
3,Jun 17 2022 8:59AM,239818,15,"Person & Covey, Inc.",21,,
4,Jun 17 2022 8:58AM,239820,10,ISDIN Corporation,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 17 2022 9:07AM,239823,19,"Graystone, LLC",2.0,NaN,NaN
1,Jun 17 2022 8:59AM,239817,10,ISDIN Corporation,60.0,NaN,NaN
2,Jun 17 2022 8:59AM,239821,12,Hush Hush,11.0,NaN,NaN
3,Jun 17 2022 8:59AM,239818,15,"Person & Covey, Inc.",21.0,NaN,NaN
4,Jun 17 2022 8:58AM,239820,10,ISDIN Corporation,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 17 2022 9:07AM,239823,19,"Graystone, LLC",2.0,0.0,0.0
1,Jun 17 2022 8:59AM,239817,10,ISDIN Corporation,60.0,0.0,0.0
2,Jun 17 2022 8:59AM,239821,12,Hush Hush,11.0,0.0,0.0
3,Jun 17 2022 8:59AM,239818,15,"Person & Covey, Inc.",21.0,0.0,0.0
4,Jun 17 2022 8:58AM,239820,10,ISDIN Corporation,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2637755,93.0,6.0,8.0
12,479640,13.0,0.0,0.0
15,1438783,70.0,0.0,1.0
16,1678427,6.0,7.0,0.0
19,719424,44.0,3.0,0.0
20,239775,15.0,0.0,0.0
21,239790,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2637755,93.0,6.0,8.0
1,12,479640,13.0,0.0,0.0
2,15,1438783,70.0,0.0,1.0
3,16,1678427,6.0,7.0,0.0
4,19,719424,44.0,3.0,0.0
5,20,239775,15.0,0.0,0.0
6,21,239790,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,93.0,6.0,8.0
1,12,13.0,0.0,0.0
2,15,70.0,0.0,1.0
3,16,6.0,7.0,0.0
4,19,44.0,3.0,0.0
5,20,15.0,0.0,0.0
6,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,93.0
1,12,Released,13.0
2,15,Released,70.0
3,16,Released,6.0
4,19,Released,44.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,8.0,0.0,1.0,0.0,0.0,0.0,0.0
Executing,6.0,0.0,0.0,7.0,3.0,0.0,1.0
Released,93.0,13.0,70.0,6.0,44.0,15.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,8.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Executing,6.0,0.0,0.0,7.0,3.0,0.0,1.0
2,Released,93.0,13.0,70.0,6.0,44.0,15.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,8.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Executing,6.0,0.0,0.0,7.0,3.0,0.0,1.0
2,Released,93.0,13.0,70.0,6.0,44.0,15.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()